In [45]:
import requests
import xmltodict
import time

LINE_ID = 1033
BUS_ID = None

In [36]:
import base64
import hashlib
from Crypto.Cipher import ARC4 as rc4

def decrypt(key, data):
    real_data = base64.b64decode(data)
    real_key = 'aibang' + key
    md5_key = bytes(hashlib.md5(real_key.encode('utf-8')).hexdigest().encode('utf-8'))
    dec = rc4.new(md5_key)
    return dec.decrypt(real_data)


In [46]:
headers = {
    'Host': 'transapp.btic.org.cn',
    'PID': '5',
    'PLATFORM': 'ios',
    'CID': '18d31a75a568b1e9fab8e410d398f981',
    'TIME': '1539706356',
    'ABTOKEN': '31d7dae1d869a172f3b66fa14fe274d1',
    'VID': '6',
    'IMEI': '3256',
    'CTYPE': 'json'
}

url = 'http://transapp.btic.org.cn:8512/ssgj/bus.php?id={id}&no=1&encrypt=1'.format(id=LINE_ID)

def get_line_info():
    global BUS_ID

    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print("request error: {code} {data}".format(code=resp.status_code, data=resp.text))
    data = xmltodict.parse(resp.text)
    bus_list = data['root']['data']['bus']
    if type(bus_list) != type(list()):
        bus_list = [bus_list]

    sync = False
    min_station = 9999999
    min_bus_id = None
    
    if BUS_ID is None:
        for bus in bus_list:
            gt = bus['gt']
            bus_id = bus['id']
            station_id = int(decrypt(gt, bus['nsn']))
            if station_id < min_station:
                min_station = station_id
                min_bus_id = bus_id
        BUS_ID = min_bus_id

    for bus in bus_list:
        gt = bus['gt']
        x = bus['x']
        y = bus['y']
        bus_id = bus['id']

        if BUS_ID != bus_id:
            continue
        
        sync = True
        x = float(decrypt(gt, x))
        y = float(decrypt(gt, y))

        return sync, x, y, gt
    return sync, BUS_ID

get_line_info()

(True, 116.421875, 39.908365, '1658764756')

In [38]:
from geopy.geocoders import Nominatim
import subprocess
import os
import zipfile


def coordinates_from_address(address):
    try:
        location = Nominatim(user_agent='changer').geocode(address)
        print("Location entered: " + location.address)
        return str(location.latitude) + ' ' + str(location.longitude)
    except:
        print('Cannot find address.')
        time.sleep(5)
        return None


def mount_image():
    cmd = 'ideviceinfo'
    version = [i for i in subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True).decode(
        ).split('\n') if i.startswith('ProductVersion')][0].split(' ')[1][:4]
    print(version)
    cmd = 'ideviceimagemounter ' + os.getcwd() + '/' + version + \
        '/DeveloperDiskImage.dmg ' + os.getcwd() + '/' + version + \
        '/DeveloperDiskImage.dmg.signature'
    try:
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Developer Disk Image mounted!')
        return True
    except Exception as e:
        if 'Device is locked' in e.output.decode() or 'Could not start' in e.output.decode():
            print('Please unlock your device and try again.')
        print(e)
        return False


def set_location(coordinates):
    try:
        cmd = 'idevicesetlocation -- ' + coordinates
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, shell=True)
        print('Device location set to ' + coordinates + '.')
    except Exception as e:
        if 'Device is locked' in e.output.decode():
            print('Please unlock your device and try again.')
            time.sleep(5)
        elif 'No device found' in e.output.decode():
            print('Please connect your device.')
            time.sleep(5)
        elif 'Make sure a developer disk image is mounted!' in e.output.decode():
            print('error')

In [5]:
mount_image()

14.0
Command 'ideviceimagemounter /Users/bytedance/Desktop/workspace/notebook/BusSync/14.0/DeveloperDiskImage.dmg /Users/bytedance/Desktop/workspace/notebook/BusSync/14.0/DeveloperDiskImage.dmg.signature' returned non-zero exit status 255.


False

In [6]:
set_location("39.0 45.0")

Device location set to 39.0 45.0.


In [7]:
import math

x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方

def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret

def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

In [49]:
import scipy.signal

now_lat = None
now_lng = None

lat = 39.966903450000004
lng = 116.33458219251584

lat_que = list()
lng_que = list()
real_lat_que = list()
real_lng_que = list()
step = 0.00005
eps = 0.00000001
xs = list()
ys = list()

def do_move():
    global lat_que, lng_que
    global now_lat, now_lng
    global real_lat_que, real_lng_que
    if len(lat_que) == 0:
        return
    lat = lat_que[0]
    lng = lng_que[0]
    if abs(lat-now_lat) + abs(lng-now_lng) < eps:
        lat_que = lat_que[1:]
        lng_que = lng_que[1:]
        return
    lat_delta = lat - now_lat
    lng_delta = lng - now_lng
    norm = (lat_delta ** 2 + lng_delta ** 2) ** 0.5

    e_step = min(step, norm)
    new_lat = now_lat + lat_delta * e_step / norm
    new_lng = now_lng + lng_delta * e_step / norm
    real_lat_que.append(new_lat)
    real_lng_que.append(new_lng)
    now_lat = new_lat
    now_lng = new_lng
    if len(real_lat_que) >= 10:
        xs.append(scipy.signal.savgol_filter(real_lat_que, 9, 3)[5])
        ys.append(scipy.signal.savgol_filter(real_lng_que, 9, 3)[5])
        real_lat_que = real_lat_que[1:]
        real_lng_que = real_lng_que[1:]
        _xs, _ys = gcj02_to_wgs84(xs[-1], ys[-1])
        set_location(str(_ys)+' '+str(_xs))

last_post = time.time()
while True:
    lat = 0.0
    lng = 0.0
    gt = 0.0
    if time.time() - last_post > 5:
        try:
            resp = get_line_info()
            if resp[0] == False:
                print("no bus info, end")
                break

            _, lat, lng, gt = resp
            print("sync with line {line}, bus {bus}, last update before {dif} sec.".format(
                line=LINE_ID, bus= BUS_ID, dif=time.time()-float(gt)))
        except Exception as e: 
            print(e)
            

        last_post = time.time()

        if now_lat is None:
            now_lat = lat
        if now_lng is None:
            now_lng = lng

        if len(lat_que) == 0 or lat != lat_que[-1]:
            lat_que.append(lat)
            lng_que.append(lng)
            print('append %f, %f' % (lat, lng))
    do_move()
    time.sleep(0.1)


sync with line 1033, bus 28377, last update before 21.63231587409973 sec.
append 116.374432, 39.907260
sync with line 1033, bus 28377, last update before 6.786664009094238 sec.
append 116.372390, 39.907290
Device location set to 39.90587704728185 116.36790700523768.
Device location set to 39.90587783771039 116.36785707689256.
Device location set to 39.905878628221444 116.36780714865064.
Device location set to 39.905879418814955 116.36775722051176.
Device location set to 39.90588020949084 116.36770729247587.
Device location set to 39.905881000249025 116.36765736454291.
Device location set to 39.90588179108945 116.36760743671273.
Device location set to 39.90588258201206 116.36755750898527.
sync with line 1033, bus 28377, last update before 12.305082082748413 sec.
Device location set to 39.90588337301675 116.36750758136044.
Device location set to 39.90588416410348 116.36745765383813.
Device location set to 39.905884955272164 116.36740772641825.
Device location set to 39.90588574652275 116

Device location set to 39.9059252773832 116.36227309254286.
Device location set to 39.905925168005666 116.36222317028255.
Device location set to 39.9059250587024 116.36217324811469.
Device location set to 39.90592494947332 116.36212332603922.
Device location set to 39.905924840318335 116.36207340405602.
Device location set to 39.905924731237405 116.36202348216497.
Device location set to 39.905924622230415 116.361973560366.
Device location set to 39.90592451329731 116.361923638659.
sync with line 1033, bus 28377, last update before 3.5401339530944824 sec.
Device location set to 39.90592440443799 116.36187371704386.
Device location set to 39.90592429565242 116.3618237955205.
Device location set to 39.90592418694048 116.36177387408878.
Device location set to 39.90592407830211 116.36172395274866.
Device location set to 39.90592396973723 116.36167403150002.
Device location set to 39.905923861245775 116.36162411034272.
Device location set to 39.90592375282765 116.3615741892767.
Device locati

Device location set to 39.90592033569342 116.35641678528793.
Device location set to 39.905920244716626 116.3563668731416.
Device location set to 39.9059201538047 116.35631696107552.
Device location set to 39.90592006295755 116.35626704908967.
Device location set to 39.9059199721751 116.35621713718385.
Device location set to 39.905919881457265 116.35616722535804.
Device location set to 39.90591979080397 116.35611731361205.
sync with line 1033, bus 28377, last update before 13.130542039871216 sec.
Device location set to 39.90591970021512 116.35606740194584.
Device location set to 39.90591960969064 116.35601749035925.
Device location set to 39.905919519230444 116.35596757885222.
Device location set to 39.90591942883445 116.3559176674246.
Device location set to 39.90591933850257 116.35586775607631.
Device location set to 39.90591924823471 116.35581784480723.
Device location set to 39.905919157869555 116.35576784435789.
Device location set to 39.905919072027345 116.35571799273703.
Device lo

Device location set to 39.905926050318406 116.35061625357498.
Device location set to 39.90592630895036 116.35057101027806.
Device location set to 39.905926460680014 116.35052619841467.
Device location set to 39.9059265019957 116.35048095503407.
Device location set to 39.90592644862865 116.35043441720708.
Device location set to 39.90592633555317 116.35038572202589.
Device location set to 39.90592621698696 116.35033400660473.
sync with line 1033, bus 28377, last update before 21.922041177749634 sec.
Device location set to 39.90592612604451 116.35028410345659.
Device location set to 39.905926035156426 116.35023420037514.
Device location set to 39.90592594432263 116.35018429736027.
Device location set to 39.905925853543025 116.35013439441187.
Device location set to 39.905925762817525 116.35008449152981.
Device location set to 39.90592567214605 116.35003458871398.
Device location set to 39.90592558152849 116.34998468596427.
Device location set to 39.905925490964776 116.34993478328056.
Devic

Device location set to 39.90592594143565 116.34490237968873.
Device location set to 39.905926006534465 116.34486446236824.
Device location set to 39.905926073403 116.34482552986186.
Device location set to 39.90592614553291 116.3447835517401.
sync with line 1033, bus 28377, last update before 11.334833860397339 sec.
Device location set to 39.90592622642223 116.34473649758104.
Device location set to 39.90592631957686 116.34468233697238.
Device location set to 39.905926410128664 116.34462971868595.
sync with line 1033, bus 28377, last update before 16.445332050323486 sec.
append 116.350647, 39.907280
sync with line 1033, bus 28377, last update before 21.591066122055054 sec.
append 116.350647, 39.907280
sync with line 1033, bus 28377, last update before 6.711282014846802 sec.
append 116.348313, 39.907298
Device location set to 39.90592645786502 116.34458163638011.
Device location set to 39.905926523427524 116.34453679424455.
Device location set to 39.905926650771164 116.34449389641273.
Dev

...

Device location set to 39.906279156947676 116.27200898073018.
Device location set to 39.90627920851777 116.27195903418774.
Device location set to 39.90627926000739 116.2719090875374.
sync with line 1033, bus 28377, last update before 25.379935026168823 sec.
Device location set to 39.906279311416476 116.27185914077903.
Device location set to 39.90627936274498 116.27180919391256.
Device location set to 39.906279413992856 116.27175924693796.
Device location set to 39.90627946516005 116.27170929985515.
Device location set to 39.90627951912353 116.27165653745553.
Device location set to 39.90627956820971 116.27160846695982.
Device location set to 39.90627961380642 116.27156374781639.
Device location set to 39.906279657292274 116.27152103946135.
Device location set to 39.90627970003786 116.27147900131993.
sync with line 1033, bus 28377, last update before 10.846137046813965 sec.
Device location set to 39.90627974340629 116.27143629280738.
Device location set to 39.906279788753196 116.27139157

Device location set to 39.906277463249126 116.26652420747078.
Device location set to 39.90627729766776 116.26647424917157.
Device location set to 39.90627713200093 116.26642429075794.
Device location set to 39.90627696624857 116.26637433222987.
Device location set to 39.906276800410666 116.26632437358731.
Device location set to 39.90627663448717 116.2662744148302.
Device location set to 39.90627646847805 116.2662244559585.
Device location set to 39.90627629121215 116.26617113779155.
Device location set to 39.90627597850235 116.26612341619452.
sync with line 1033, bus 28377, last update before 15.784852981567383 sec.
Device location set to 39.906275781676115 116.26607969483291.
Device location set to 39.9062758787279 116.26603837642932.
Device location set to 39.906276374320775 116.26599786276564.
Device location set to 39.90627729978735 116.26595655468431.
Device location set to 39.906278613129736 116.26591285208825.
Device location set to 39.90628019901866 116.26586515393969.
Device l

Device location set to 39.90635975647597 116.26069349603941.
Device location set to 39.90636043464609 116.26064613887067.
Device location set to 39.90635834109677 116.26060027623983.
Device location set to 39.90635241400163 116.26055539247034.
Device location set to 39.906342301187074 116.26051085462566.
Device location set to 39.90632836013152 116.26046591250818.
Device location set to 39.90631165796439 116.26041969865784.
Device location set to 39.90629397146486 116.26037122835059.
sync with line 1033, bus 28377, last update before 27.478145122528076 sec.
Device location set to 39.90627776960255 116.26032396400309.
Device location set to 39.9062615676599 116.26027669954821.
Device location set to 39.906245365636885 116.2602294349859.
Device location set to 39.90622916353349 116.26018217031614.
Device location set to 39.90621296134969 116.26013490553889.
Device location set to 39.90619675908545 116.26008764065412.
Device location set to 39.90618055674074 116.26004037566179.
Device loc

Device location set to 39.90449506909765 116.25512422348037.
Device location set to 39.904478858178926 116.25507694703398.
Device location set to 39.9044626471775 116.25502967047716.
Device location set to 39.904446436093345 116.25498239380987.
Device location set to 39.904430224926415 116.25493511703209.
Device location set to 39.90441401367672 116.25488784014381.
Device location set to 39.90439780234425 116.254840563145.
sync with line 1033, bus 28377, last update before 14.71640920639038 sec.
Device location set to 39.904381590928956 116.25479328603562.
Device location set to 39.904365379430864 116.25474600881567.
Device location set to 39.90434916784992 116.25469873148513.
Device location set to 39.90433295618613 116.25465145404398.
Device location set to 39.90431674443946 116.25460417649218.
Device location set to 39.90430053260991 116.25455689882972.
Device location set to 39.90428432069745 116.25450962105658.
Device location set to 39.90426810870207 116.25446234317273.
Device lo

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(ys, xs)
plt.show()